In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import os
import pandas as pd
from PIL import Image

# ─── 커스텀 Dataset 클래스 ─────────────────────────────
class ApparelDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        csv_file: CSV 파일 경로 (train.csv, val.csv 등)
        root_dir: 이미지가 저장된 루트 디렉토리
                  예: "C:\\Users\\Admin\\Desktop\\data\\apparel-image-dataset-2\\clothes_dataset"
        transform: 이미지 변환 함수
        
        CSV 파일 포맷:
          - 첫 번째 열: 인덱스 (사용하지 않음)
          - 두 번째 열: 이미지 파일명 (예: "./clothes_dataset/red_dress/xxx.jpg" 또는 ".\\clothes_dataset\\red_dress\\xxx.jpg")
          - 세 번째 열부터: 11개의 라벨 (0 또는 1)
        """
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # CSV의 두 번째 열에서 이미지 파일명을 읽습니다.
        img_filename = str(self.data.iloc[idx, 1]).strip()
        # 백슬래시를 슬래시로 변경하여 경로 형식을 통일합니다.
        img_filename = img_filename.replace("\\", "/")
        # 만약 파일명이 "./clothes_dataset/"로 시작하면 해당 접두어를 제거합니다.
        prefix = "./clothes_dataset/"
        if img_filename.startswith(prefix):
            img_filename = img_filename[len(prefix):]
        # 최종 경로: root_dir과 img_filename을 결합
        img_path = os.path.join(self.root_dir, img_filename)
        
        # 이미지 열기 (RGB 모드로 변환)
        image = Image.open(img_path).convert("RGB")
        # 세 번째 열부터 11개의 라벨을 float32 배열로 변환
        labels = self.data.iloc[idx, 2:].values.astype('float32')
        
        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(labels)

- __init__ 메서드:
  - CSV 파일을 읽어서 self.data에 저장합니다.
  - root_dir는 이미지 파일들이 저장된 최상위 폴더 경로입니다.
  - transform은 이미지 전처리(예: 리사이즈, 정규화) 함수입니다.

- __len__ 메서드:
  - 데이터셋의 길이(샘플 개수)를 반환합니다.

- __getitem__ 메서드:
  - 지정한 인덱스의 이미지 파일명을 CSV에서 가져와 문자열로 변환합니다.
  - 경로의 백슬래시를 슬래시로 변환하고, 접두어("./clothes_dataset/")가 있다면 제거합니다.
  - 최종 경로를 os.path.join을 통해 생성한 후, 이미지를 열어 RGB로 변환합니다.
  - 이후 CSV의 나머지 열에서 11개 라벨을 읽어 텐서로 변환하여 반환합니다.

In [22]:
# ─── 경로 및 CSV 파일 설정 ─────────────────────────────
data_dir = "C:\\Users\\Admin\\Desktop\\data\\apparel-image-dataset-2\\clothes_dataset"
train_csv = "C:\\Users\\Admin\\Desktop\\data\\apparel-image-dataset-2\\train.csv"
val_csv = "C:\\Users\\Admin\\Desktop\\data\\apparel-image-dataset-2\\val.csv"
test_csv = "C:\\Users\\Admin\\Desktop\\data\\apparel-image-dataset-2\\test.csv"

In [23]:
# ─── 데이터 전처리 ─────────────────────────────
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

- Resize: 모든 이미지를 224x224 픽셀 크기로 맞춥니다.
- ToTensor: 이미지를 파이토치 텐서로 변환합니다.
- Normalize: 사전 학습된 모델(ImageNet 기준)에 맞게 정규화합니다.

In [24]:
# ─── Dataset 및 DataLoader 생성 ─────────────────────────────
train_data = ApparelDataset(csv_file=train_csv, root_dir=data_dir, transform=transform)
val_data = ApparelDataset(csv_file=val_csv, root_dir=data_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

- ApparelDataset 객체를 생성하여 훈련 데이터와 검증 데이터를 만듭니다.
- DataLoader는 데이터를 배치 단위로 로드하며, 훈련 시 셔플(shuffle)을 활성화합니다.

In [25]:
# ─── 사전 학습된 ResNet-18 모델 불러오기 및 전이학습 적용 ─────────────────────────────
model = models.resnet18(pretrained=True)

c:\Users\Admin\miniconda3\envs\myGPU\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Admin\miniconda3\envs\myGPU\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
# 전이학습: 모든 파라미터 동결 (freeze)
for param in model.parameters():
    param.requires_grad = False

# 마지막 fc 레이어 수정: 출력 클래스 수를 11로 변경
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 11)

# 마지막 fc 레이어의 파라미터만 학습하도록 설정
for param in model.fc.parameters():
    param.requires_grad = True

- 모든 파라미터 동결:
  - 모델의 모든 파라미터에 대해 requires_grad를 False로 설정하여 기존 사전 학습된 가중치가 업데이트되지 않도록 합니다.

- 마지막 fc 레이어 수정:
  - 기존 모델의 마지막 fully connected 레이어를 새로운 데이터셋의 클래스 수(11)로 맞추기 위해 재정의합니다.

- fc 레이어 파라미터 학습 활성화:
  - 새로 정의한 fc 레이어의 파라미터에 대해서만 requires_grad를 True로 설정하여 학습이 진행되도록 합니다.

In [27]:
# ─── 모델을 GPU로 이동 ─────────────────────────────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [32]:
# ─── 손실 함수 및 옵티마이저 설정 ─────────────────────────────
# BCEWithLogitsLoss는 내부에서 sigmoid를 적용한 후 이진 교차 엔트로피 손실을 계산함.
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ─── 모델 훈련 함수 ─────────────────────────────
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        # 훈련 루프: 각 배치에 대해 손실을 계산하고, 역전파 수행
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)  # 모델의 raw logits 출력
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)

        # 평가 루프: 검증 데이터셋에 대해 모델 평가
        model.eval()
        correct_preds = 0
        total_preds = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                # 평가 시, sigmoid를 적용하여 확률로 변환 후, 0.5 임계값을 기준으로 예측
                preds = (torch.sigmoid(outputs) > 0.5).float()
                correct_preds += (preds == labels).sum().item()
                total_preds += labels.size(0) * labels.size(1)

        epoch_acc = correct_preds / total_preds
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
        
    return model

- 손실 함수:
  - BCEWithLogitsLoss는 마지막 레이어의 raw logits(활성화 함수 없이 출력된 값)에 대해 내부에서 sigmoid를 적용하여 이진 교차 엔트로피 손실을 계산합니다.

- 옵티마이저:
  - Adam 옵티마이저를 사용하여 모델 파라미터를 업데이트합니다.

- 훈련 루프:
  - 각 에포크마다 훈련 데이터에 대해 모델을 학습시키며, 배치별 손실을 누적하여 에포크 손실을 계산합니다.

- 평가 루프:
  - 검증 데이터를 사용하여 모델을 평가합니다.
  - 평가 시에는 raw logits에 대해 torch.sigmoid를 적용하고, 0.5 임계값을 사용하여 각 클래스의 예측(0 또는 1)을 결정합니다.

- 에포크별 출력:
  - 에포크마다 평균 손실과 정확도를 출력합니다.

In [33]:
# ─── 모델 훈련 시작 ─────────────────────────────
trained_model = train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Loss: 0.0739, Accuracy: 0.9749
Epoch 2/10, Loss: 0.0695, Accuracy: 0.9760
Epoch 3/10, Loss: 0.0681, Accuracy: 0.9775
Epoch 4/10, Loss: 0.0660, Accuracy: 0.9775
Epoch 5/10, Loss: 0.0669, Accuracy: 0.9770
Epoch 6/10, Loss: 0.0649, Accuracy: 0.9769
Epoch 7/10, Loss: 0.0635, Accuracy: 0.9771
Epoch 8/10, Loss: 0.0638, Accuracy: 0.9759
Epoch 9/10, Loss: 0.0620, Accuracy: 0.9771
Epoch 10/10, Loss: 0.0620, Accuracy: 0.9775


In [34]:
# ─── 훈련된 모델 저장 ─────────────────────────────
torch.save(trained_model.state_dict(), 'apparel_resnet_model.pth')